Import the necessary Python packages - particularly rpy2.

In [1]:
import numpy as np
import scipy as sp
from rpy2.robjects.packages import importr # You may need to run "pip install rpy2" in Terminal
import rpy2 as ro
import pandas as pd
import datetime as dt

In [3]:
from stitch.query import Redshift as rs

benjamins-air.sf.stch.co


In [8]:
with rs.RedshiftConnection(
    user='############',password='#################') as conn:
    conn.execute("set search_path = 'data_warehouse';")
    query1="""
        SELECT DISTINCT(a.ts) as ts, (arr/12) as mrr
        FROM
        (
            SELECT DISTINCT DATE(created_at) as ts
            FROM activity_account
        ) a
        LEFT  JOIN
        (
            SELECT log_ymd as ts, arr, deleted_at
            FROM account_financial_mrr_log
            WHERE EXTRACT(month FROM cohort_year_month_date) = EXTRACT(month FROM log_ymd)
            AND cohort_year_month_date >= '2015-08-01' 
            AND cohort_year_month_date <= DATE(convert_timezone('GMT','US/Pacific',CURRENT_DATE))
        ) b
        ON a.ts = b.ts
        WHERE a.ts >= '2015-08-01'
        AND deleted_at IS NULL
        ORDER BY ts
    """
    df = conn.query(query1)

NameError: name 'rs' is not defined

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R -i df
df

In [ ]:
%R plot(df$mrr)

With outliers removed, we have 72 missing observations.

In [ ]:
%%R 
df$mrr[df$mrr < 200000] <- NA
df$mrr[df$mrr > 510110.7] <- NA
sum(is.na(df$mrr)) # ... and count missing values

In [ ]:
%%R -o df # Export the R object "df" into python...
#install.packages('imputeTS', repos='http://cran.us.r-project.org')
library('imputeTS')
ts <- ts(df$mrr)
df$mrr <- na.interpolation(ts, option ="spline")

After interpolating the missing data, we can use the "tseries" package from CRAN to test for stationarity.

In [ ]:
%%R -i df # Import the Python object "df" into R...
#Check the stationary assumption
#install.packages('tseries', repos='http://cran.us.r-project.org')
library('tseries')
test <- ts(df$mrr)
adf.test(test)

In [ ]:
%%R -i df # Import the Python object "df" into R...
# Take the first differences
df_diff <- diff(df$mrr,1)
plot(df_diff, type='l')

In [ ]:
%%R -i df # Import the Python object "df" into R...
#install.packages('astsa', repos='http://cran.us.r-project.org')
library(astsa)
acf2(df_diff, 40)

The package necessary for constructing a periodgram can be found in the "TSA" package from CRAN.
See: https://cran.r-project.org/web/packages/TSA/TSA.pdf

In [ ]:
# from rpy2.robjects.packages import importr
# utils = importr('utils')
# utils.install_packages('TSA')
# r.packages.utils.install_packages(package_name)

A periodogram of the data concurs with the ACF and PACF and confirms the absence of seasonlity. 

In [ ]:
%%R # Import the Python object "df" into R...
# install.packages('TSA', repos='http://cran.us.r-project.org')
library('TSA')
test <- ts(df$mrr)
periodogram(test,ylab='Variable Star Periodogram');  abline(h=0)

In [ ]:
%%R -i df # Import the Python object "df" into R...
#install.packages('forecast', repos='http://cran.us.r-project.org')
library(forecast)
# compel the auto.arima to assume no seasonality, 7-day seasonality, and 30-day seasonality
ts_0 <- ts(df$mrr) 
# ts_1 <- ts(df$mrr,frequency=7) 
# ts_2 <- ts(df$mrr,frequency=30) 
fit0 <- auto.arima(ts_0, trace=TRUE, allowdrift=FALSE, ic="aicc", seasonal=TRUE, lambda=0) 
# fit1 <- auto.arima(ts_1, trace=TRUE, allowdrift=TRUE, ic="bic", seasonal=TRUE, lambda=0) 
# fit2 <- auto.arima(ts_2, trace=TRUE, allowdrift=TRUE, ic="bic", seasonal=TRUE, lambda=0)
# # other ic options are "aicc" and "bic"
# fit0
# fit1
# fit2

In [ ]:
%%R -i df # Import the Python object "df" into R...
# http://artax.karlin.mff.cuni.cz/r-help/library/TSA/html/tsdiag.Arima.html
# model1 <- Arima(df$mrr, order=c(1,1,2),lambda=0, seasonal = list(order = c(0, 0, 0), period = 0))
# model1 <- Arima(df$mrr, order=c(1,1,2),lambda=0)
# model1
# model2 <- Arima(df$mrr, order=c(1,1,1),lambda=0)
# model2
model3 <- Arima(df$mrr, order=c(2,1,1),lambda=0)
model3
# model4 <- Arima(df$mrr, order=c(0,1,0),lambda=0)
# model4

In [ ]:
%%R
png(filename="Forecast_Diagnostics.png")
tsdiag(model3, tol = 0.1, omit.initial = TRUE, col = "red")
dev.off()

In [ ]:
%%R -i df # Import the Python object "df" into R...
# install.packages('lubridate', repos='http://cran.us.r-project.org')
require(lubridate)
days_until_ye <- (365 - yday(as.Date(Sys.Date())))
days_until_ye

In [ ]:
%%R -i df # Import the Python object "df" into R...
# install.packages('forecast', repos='http://cran.us.r-project.org')
# https://cran.r-project.org/web/packages/forecast/forecast.pdf
library('forecast')

In [ ]:
%%R
png(filename="MRR_Forecast.png")
mar.default <- c(5,4,4,2) + 0.1
par(mar = mar.default + c(0, 4, 0, 0)) 
plot(forecast(model3, h=120), main="Projected MRR (60 Days Out)", xlab="Month", yaxt='n', xaxt='n') 
abline(v = 31, col = "grey") # August 1st, 2015
abline(v = 63, col = "grey") # September 1st, 2015
abline(v = 92, col = "grey") # October 1st, 2015
abline(v = 123, col = "grey") # November 1st, 2015
abline(v = 153, col = "grey") # December 1st, 2015
abline(v = 184, col = "grey") # January 1st, 2016
abline(v = 215, col = "grey") # February 1st, 2016
abline(v = 244, col = "grey") # March 1st, 2016
abline(v = 275, col = "grey") # April 1st, 2016
abline(v = 305, col = "grey") # May 1st, 2016
abline(v = (yday(as.Date(Sys.Date())) + 184), col = "black", lwd=2) # May 1st, 2016
abline(h = 470496.54, col = "red", lty=2) # January 2016 MRR Target
abline(h = 511009.73, col = "red", lty=2) # February 2016 MRR Target
abline(h = 551163.49, col = "red", lty=2) # March 2016 MRR Target
axis(2, at=axTicks(2), labels=sprintf("$%s", axTicks(2)), las=1)
axis(1, at = c(0,63,123,184,244,305), 
        labels = c("07/01/15","09/01/15","11/01/15","01/01/16","03/01/16","05/01/16"), 
        las=1)
# legend(0,585000, c('Point Estimates','80% C.I.','95% C.I.'), lty=c(1,1), 
#        lwd=c(2.5,2.5),col=c('navy', 'slategray',"slategray2")) 
text(50, 480000, "January Target")
text(125, 520000, "February Target")
text(185, 560000, "March Target")
# the "las" option specifies the axis label orientation, i.e. (0=parallel, 1=all horizontal, 
# 2=all perpendicular to axis, 3=all vertical)
dev.off()

In [ ]:
%%R
output_df <- as.data.frame(forecast(model3, h=90))
output_df$forecasted_for <- seq(as.Date("2016-03-18"), as.Date("2016-06-15",), by = "days") + 719163
output_df$forecasted_on <- as.Date("2016-03-17") +719163
library(plyr)
output_df <- rename(output_df, c("Point Forecast" = "point_forecast"))
output_df <- rename(output_df, c("Lo 80" = "lo_80", "Hi 80" = "hi_80"))
output_df <- rename(output_df, c("Lo 95" = "lo_95", "Hi 95" = "hi_95"))
as.numeric(output_df$forecasted_on)

as.Date("1969-01-01") - as.Date("0000-01-01")


In [ ]:
%R -o output_df

In [ ]:
output_df

In [ ]:
output_df.forecasted_for = output_df.forecasted_for.astype(int)
output_df.forecasted_on = output_df.forecasted_on.astype(int)

In [ ]:
output_df.forecasted_on = output_df.forecasted_on.apply(dt.date.fromordinal)
output_df.forecasted_for = output_df.forecasted_for.apply(dt.date.fromordinal)

In [ ]:
output_df.head()

In [ ]:
# output_df.to_csv('output_df.csv')

In [ ]:
# output_df = pd.read_csv('output_df.csv')

In [ ]:
# output_df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
# reload(rs)

In [ ]:
def insert_mrr_predictions():
    schema='data_team'
    tablename='mrr_forecast'
    service='csv_import'
    
    #/*output_df*/
    with rs.RedshiftConnection(user='###########',password='######################') as insert_conn:
        insert_conn.jobAuditor.start(schema=schema,table=tablename,service=service,status='ok')
        insert_conn.jobAuditor.log('initialize',True)
        insert_conn.jobAuditor.log('max_record_at',dt.datetime.today())
        #initialize = True - > truncates table and then inserts - start with a fresh table
        #initialize = False - > appends
        insert_conn.insert(schema,tablename,output_df,log=True,initialize=False,vacuum=True)

In [ ]:
insert_mrr_predictions()